<a href="https://colab.research.google.com/github/mcppp/FinalProjectI2DL/blob/main/LIVEtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import csv
import numpy as np
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

from PIL import Image 

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torch.optim.lr_scheduler import StepLR

print(torch.__version__)
import cv2
import numpy as np

1.7.1


In [ ]:
data_path = './FER2013' #make sure path is correct 

In [ ]:
class FERdataset(Dataset):
    def __init__(self, data_path, is_training):
        self.data_path = data_path
        self.train_path = os.path.join(data_path, 'train') #directory path for train set
        self.val_path = os.path.join(data_path, 'validate') #directory path for validation set
        self.is_training = is_training
        if self.is_training:  #based on flag is_training, the target_path will be assigned
            self.target_path = self.train_path
        else:
            self.target_path = self.val_path

        self.classes = sorted(os.listdir(self.target_path)) 
        self.img_path_label = list() #creating empty list 
        for c in self.classes: #for all classes found in directory
            img_list = os.listdir(os.path.join(self.target_path, c)) #list of img names found in target_path
            for fp in img_list: #for each img in each class folder
                full_fp = os.path.join(self.target_path, c, fp)  #save full path of each image
                self.img_path_label.append((full_fp, c, self.classes.index(c))) #fill/make list of full file path of img, class name, index of class required to train nn
    
        self.tensor_transform = torchvision.transforms.ToTensor() #converts img to tensor type
               
    def __len__(self):
        return len(self.img_path_label)

    def __getitem__(self, idx):
        (fp, class_name, class_label) = self.img_path_label[idx]
        img = Image.open(fp)
        original_img = self.tensor_transform(img)  #make original img into tensor

        input = self.tensor_transform(img) #applying defined transformations to validation data
            
        sample = dict()   #creating dict
        sample['input'] = input #our transformed image
        sample['original_img'] = original_img  #original image
        sample['target'] = class_label
        sample['class_name'] = class_name

        return sample

In [ ]:
train_dataset = FERdataset(data_path, True)
val_dataset = FERdataset(data_path, False)


In [ ]:
torch.manual_seed(0)
val_set, test_set = torch.utils.data.random_split(val_dataset, [3589, 3589])

In [ ]:
batch_size = 64

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True) #DataLoader is for batching, etc. (sampling)

val_dataloader = DataLoader(val_set, batch_size=1, shuffle=False, pin_memory=True)

test_dataloader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

num_classes = 7

In [ ]:
 device = 'cpu'
#device = 'cuda'
#print('Current Device : {}'.format(device))

In [ ]:
#how to import model from torch
class Model(nn.Module): 
    def __init__(self, feat_dim = 2048, dim_output=7):
        super(Model, self).__init__()

        self.feat_dim = feat_dim #dim of feature after getting flattened into a vector (we check this, we do not know it in advance)
        self.dim_output = dim_output #output dimension = number of classes
        

        ###################################### PRE-TRAINED MODEL ###########################################
        self.backbone = torchvision.models.resnet152(pretrained=True) #download model that's already been trained based on image classification problem
        
        for p in list(self.backbone.children())[:-1]:
            p.requires_grad = False  #parameters of layers inside list in line above become fixed
                                     #True: parameter will be trained, False: parameter won't be trained, frozen
          
        # # # get the structure until the Fully Connected Layer
        modules = list(self.backbone.children())[:-1]  #list of all layers except last one (the fc one)
        self.backbone = nn.Sequential(*modules) #put list into nn mode, not just a list -> this is our new model without last layer

        
        ######################################## DEFINING MY NEW LAYERS ####################################################
        self.fc1 = nn.Linear(feat_dim, feat_dim//2) # 2048 -> 1024 
                                                    # we know input has to be 2048 because that was the input of the fc we deleted from pre-trained model
        self.fc2 = nn.Linear(feat_dim//2, feat_dim//4) # 1024 -> 512
        self.fc3 = nn.Linear(feat_dim//4, dim_output) # 512 -> 7 must be the output dim, it is the number of classes (pokemon types)
       
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.LeakyReLU(0.1, inplace=True)

        
        ######################################## STRUCTURE OF FINAL MODEL ####################################################

    def forward(self, img):  
        batch_size = img.shape[0] 
        out = self.backbone(img)       
        out = out.view(batch_size, -1)      
        out = self.fc1(out)
        out = self.dropout(self.relu(out)) 
        out = self.fc2(out) 
        out = self.dropout(self.relu(out)) 
        out = self.fc3(out) 

        return out

In [ ]:
model = Model() # creating model
model = model.to(device) # to train on GPU

optimizer = optim.AdamW(model.parameters(), lr=1e-4) #define optimizer


In [ ]:
scheduler = StepLR(optimizer, step_size=4, gamma=0.1) #learning rate scheduler

In [ ]:
#we only need validation function for reporting on unseen test set (we are not training in this notebook)

def validate(model, sample):
    model.eval()  #turning on evaluatino time

    criterion = nn.CrossEntropyLoss() #loss function

    with torch.no_grad(): # turning off gradient computation when we know we won't call tensor.backward()

        inp = np.repeat(sample['input'].squeeze()[..., np.newaxis], 3, -1)
        inp = torch.Tensor(inp)
        inp = torch.unsqueeze(inp, 0)
        inp = inp.permute(0,3,1,2)        
       
        input=inp.float().to(device) # our transformed image batch; torch.Size([64, 3, 224, 224])
        target = sample['target'].long().to(device) # class label batch; torch.Size([64])

        pred = model(input)
        pred_loss = criterion(pred, target)

        top3_val, top3_idx = torch.topk(pred, 3)

        num_correct = torch.sum(top3_idx == target.view(-1, 1))

    return pred_loss.item(), num_correct.item(), pred

In [ ]:
#loading what we already trained
best_path=torch.load('./face_11.pth',map_location=torch.device('cpu'))
model.load_state_dict(best_path['model_state_dict'])

<All keys matched successfully>

In [ ]:
# Reporting accuracy on test set
test_loss = 0.0
test_accu = 0.0

    # Iterate over the val_dataloader
with tqdm(total=len(test_dataloader)) as pbar: # SUB PROGRESS BAR 2, up to 586 because that's the number of images in the 1 batch of val_dataloader
    for idx, sample in enumerate(test_dataloader): # for each of the 586 images
            curr_loss, num_correct,_ = validate(model, sample)
            test_loss += curr_loss / len(test_dataloader) # average validation loss
            test_accu += num_correct / len(test_dataloader) # average accuracy
            pbar.update(1) # update SUB PROGRESS BAR 2
            
print(test_loss, test_accu)


1.1537466278913924 0.8988576205070934


In [ ]:
print('Reported test accuracy: ', test_accu)

Reported test accuracy:  0.8988576205070934


In [ ]:
##Live testing

results={0:'angry', 1:'disgust', 2:'fear', 3:'happy', 4:'neutral', 5:'sad', 6:'surprise'}

rect_size = 4
cap = cv2.VideoCapture(0) 

cascPath = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
haarcascade = cv2.CascadeClassifier(cascPath)

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
     
        rerect_sized=cv2.resize(face_img,(48,48))
  
        normalized=rerect_sized/255.0 
        
        rgb_weights = [0.2989, 0.5870, 0.1140]
        gray = np.dot(normalized[...,:3], rgb_weights)
        gray_tripled = np.repeat(gray[..., np.newaxis], 3, -1)
        
        gray_tripled = torch.Tensor(gray_tripled)
        unorg = torch.unsqueeze(gray_tripled, 0)
        org = unorg.permute(0,3,1,2)  
              
        result=model(org.float())

        label1=torch.argmax(result, dim=1)[0]
        label=label1.detach().item()
      
        cv2.rectangle(im,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'